### 데이터 값 찾기

* 문서 URL 목록 가져오기

In [ ]:
import pandas as pd

# 사업보고서 제무제표 주석 문서 URL 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes_urls/fs_notes_url_all.csv"
fs_notes_all = pd.read_csv(file_path, dtype=object)
# 회사코드 정렬
fs_notes_all = fs_notes_all.sort_values(by=['corp_code'], ascending=True)
# 회사코드 목록
list_all_codes = fs_notes_all['corp_code'].unique()
len(list_all_codes)

342

* 이미 데이터 값을 찾은 것은 대상에서 제외

In [54]:
import os.path

# 이미 데이터 값을 찾은 회사코드 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/items/items_all.csv"
 
if os.path.exists(file_path):
    try:
        items_all = pd.read_csv(file_path, dtype=object)
        list_own_codes = items_all['corp_code'].unique()
    except pd.errors.EmptyDataError:
        list_own_codes = []    
else:
    list_own_codes = []
    
len(list_own_codes)

0

In [55]:
# 리스트에서 중복 제거 (이미 corp info가 있는 경우 대상에서 제외)
list_target_codes = list(set(list_all_codes) - set(list_own_codes))
len(list_target_codes)

342

* 리스트 분할

In [60]:
list_cnt = len(list_target_codes)
n = 100
list_of_lists = [list_target_codes[i * n:(i + 1) * n] for i in range((list_cnt + n - 1) // n )] 
len(list_of_lists)

4

* 항목과 값 찾기: 확정급여채무의 현재가치, 사외적립자산의 공정가치

In [61]:
import traceback
from time import sleep
import requests
from bs4 import BeautifulSoup
import re

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil, mydart

# USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
# headers = {'User-Agent': USER_AGENT}

# # 금액 단위 찾기
# def get_unit(text):
#     if '(단위:백만원)' in text or '[단위:백만원]' in text: unit = '백만원'
#     elif '(단위:천원)' in text or '[단위:천원]' in text: unit = '천원'
#     elif '(단위:원)' in text or '[단위:원]' in text: unit = '원'
#     else: unit = None   
#     return unit

# # 콤마와 괄호를 제거한다.
# def str2num(x):
#     if "," in x: x = x.replace(",","")
#     if "(-)" in x: x = x.replace("(-)","")    
#     if x.startswith("("): x = x.lstrip("(")
#     if x.endswith(")"): x = x.rstrip(")")
#     return float(x)

# # 재무제표 주석 문서에서 사용된 금액 단위 찾기
# def find_unit(doc_url):
#     unit = None
#     response = requests.get(doc_url)
#     if response.status_code == 200:     # URL GET '200 정상'
#         soup = BeautifulSoup(response.text)
#         doc_text = soup.text
#         for a in re.finditer(r"확정급여+[가-힣]+의 현재가치", doc_text):
#             sliced_text = doc_text[a.start()-100:a.start()]
#             trimmed_text = sliced_text.replace(" ","")     # 문서에 있는 모든 공백 제거
#             unit = get_unit(trimmed_text)
#             if unit != None:
#                 return unit
#         for a in re.finditer("퇴직급여채무", doc_text):
#             sliced_text = doc_text[a.start()-100:a.start()]
#             trimmed_text = sliced_text.replace(" ","")     # 문서에 있는 모든 공백 제거
#             unit = get_unit(trimmed_text)
#             if unit != None:
#                 return unit     
#     else:
#         unit = None   
#     return unit    

# # 재무제표 주석 문서에서 특정 항목의 금액 찾기
# def find_item(doc_url):
#     val_a = val_b = 0.0                    
#     doc_data = pd.read_html(doc_url)
#     for df in doc_data:
#         row = df.shape[0]
#         for i in range(0, row):
#             item = str(df.iloc[i,0])
#             if item.__contains__('확정급여채무의 현재가치') or item.__contains__('확정급여부채의 현재가치') or item.__contains__('퇴직급여채무'):
#                 # print(i, type(df.iloc[i,1]), df.iloc[i,1])
#                 str_a = str(df.iloc[i,1])
#                 if str_a == "-": val_a = 0.0
#                 else: val_a += str2num(str_a)       # 값을 합한다.
#             elif item.__contains__('사외적립자산의 공정가치'):
#                 # print(i, type(df.iloc[i,1]), df.iloc[i,1])                            
#                 str_b = str(df.iloc[i,1])
#                 if item == str_b: val_b = -1        # 값 찾기 오류
#                 elif str_b == "-": val_b = 0.0
#                 else: val_b = str2num(str_b)
#             else:
#                 continue    
#         if (val_a != 0.0) and (val_b != 0.0):
#             break   
#     return val_a, val_b       

In [62]:
from tqdm import tqdm

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items/"

for corp_codes in list_of_lists:
    for corp_code in tqdm(corp_codes):
        str_expr = "corp_code == @corp_code"
        df_doc = fs_notes_all.query(str_expr)
        doc_count = df_doc.shape[0]
        data_list = []
        for i, doc in df_doc.iterrows():
            sleep(1.0)
            try:
                val_a = val_b = 0.0
                unit = None
                doc_result = doc['doc_result']
                if doc_result == 'No Financial Statement Notes':
                    data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                    'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                    'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'unit' : unit, 
                                    'val_a' : None, 'val_b' : None, 'item_result' : None})
                else:
                    # 재무제표 주석 문서에서 사용된 금액 단위 찾기 (확정급여채무)
                    unit = mydart.find_unit(doc['doc_url'])
                    # 당기말의 "확정급여채무"와 "사외적립자산" 금액 찾기
                    val_a, val_b = mydart.find_item(doc['doc_url'])      
                    data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                    'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                    'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'unit' : unit, 
                                    'val_a' : val_a, 'val_b' : val_b, 'item_result' : 'OK'})
            except ValueError as err:
                print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc['doc_url'])
                print(err)
                break
            except Exception as e:
                print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc['doc_url'])
                err_msg = traceback.format_exc()
                print(err_msg)
                break
        # 결과를 파일로 저장
        doc_value_count = len(data_list)
        if doc_value_count == 0:
            break
        elif doc_value_count == doc_count:        
            df_items = pd.DataFrame(data_list)
            file_name = corp_code + '_item_' + myutil.now_dt_str() + '.csv'
            df_items.to_csv(path_dir + file_name, index=False)
        else:
            continue

 53%|█████▎    | 53/100 [14:55<12:12, 15.58s/it]

00112022 대원화성 20180402001524 사업보고서 (2017.12) http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402001524 5. 재무제표 주석 http://dart.fss.or.kr/report/viewer.do?rcpNo=20180402001524&dcmNo=6047053&eleId=16&offset=856727&length=413319&dtd=dart3.xsd
Traceback (most recent call last):
  File "C:\Users\kyungho\AppData\Local\Temp\ipykernel_16040\3851240639.py", line 27, in <module>
    val_a, val_b = find_item(doc['doc_url'])
  File "C:\Users\kyungho\AppData\Local\Temp\ipykernel_16040\805333181.py", line 64, in find_item
    str_a = str(df.iloc[i,1])
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\core\indexing.py", line 1096, in __getitem__
    return self.obj._get_value(*key, takeable=self._takeable)
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\core\frame.py", line 3867, in _get_value
    series = self._ixs(col, axis=1)
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\core\frame.py", line 36

 53%|█████▎    | 53/100 [14:57<13:15, 16.93s/it]


00218575 황금에스티 20220321000623 사업보고서 (2021.12) http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220321000623 5. 재무제표 주석 http://dart.fss.or.kr/report/viewer.do?rcpNo=20220321000623&dcmNo=8482141&eleId=22&offset=722699&length=289985&dtd=dart3.xsd
could not convert string to float: '<퇴직급여채무>'


 36%|███▌      | 36/100 [09:55<23:07, 21.68s/it]

00145552 이수화학 20181107000303 [기재정정]사업보고서 (2017.12) http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20181107000303 5. 재무제표 주석 http://dart.fss.or.kr/report/viewer.do?rcpNo=20181107000303&dcmNo=6372797&eleId=18&offset=1100466&length=412903&dtd=dart3.xsd
Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connection.py", line 200, in _new_conn
    sock = connection.create_connection(
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv

 39%|███▉      | 39/100 [11:38<18:11, 17.90s/it]


KeyboardInterrupt: 